# Assignment 5

In [53]:
import numpy as np
import pandas as pd 
import scipy as sci
import math 
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi, cos, sin, exp, sqrt
from scipy.signal import freqz, welch, periodogram, butter, lfilter, filtfilt, boxcar, ellip

from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import FixedFormatter

from mpl_toolkits.basemap import Basemap
from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import data

In [2]:
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=[0, 1])

DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

### Clean up data

In [3]:
# Clean up minute resolution data 
# -------------------------------

def DateInsert(Data):
    date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     freq='min', periods = len(Data))
    # Insert dates into D2 dataframe
    Data.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D1 = Data.rename(index=str, columns={Data.columns[1]: "Temperature"})
    # Set index
    DM = D1.set_index('Time')
    return DM

DataM = [DC, JB]
DataMLabels = ['Deep Cove', 'James Bay']

# Insert dates
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

In [4]:
# Clean up hour resolution data
# -----------------------------

# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN', 'NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, columns={"NaN": "MatLab Time"})

# Set time as index column
AS3 = AS2.set_index('Time')

# Get the longitudes correct 
for i in range(1, np.shape(AS3)[1]):
    New_name = float(list(AS3)[i][1]) - 360
    AS3 = AS3.rename(columns={list(AS)[i][1]: str(format(New_name, '.4f'))})

AS4 = AS3.drop('MatLab Time', axis=1, level=1)
    
AS = AS4['48.4623']['-123.3090']

### Select data time

In [5]:
# Select dates
DCs = DataMIns[0].loc['2015-06-01 00:00':'2015-09-30 23:59']['Temperature']
JBs = DataMIns[1].loc['2015-06-01 00:00':'2015-09-30 23:59']['Temperature']
DCw = DataMIns[0].loc['2014-11-01 00:00':'2015-02-28 23:59']['Temperature']
JBw = DataMIns[1].loc['2014-11-01 00:00':'2015-02-28 23:59']['Temperature']

US = AS.loc['2016-01-01 00:00':'2016-12-30 23:59']

## Question 1: Power within a frequency band 

In [39]:
steps3 = 3*10**4
range3 = 15

In [40]:
# Plot out the time series 
# ========================

fig, (DCs_Plot, JBs_Plot, DCw_Plot, JBw_Plot) = plt.subplots(4, 1, figsize=(12, 16))

DCs_Plot.plot(DCs, linewidth=1)
DCw_Plot.plot(DCw, linewidth=1)
JBs_Plot.plot(JBs, linewidth=1)
JBw_Plot.plot(JBw, linewidth=1)

DCs_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

DCs_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

DCs_Plot.set_title("Minute resolution measuerments of temperature at Deep Cove from 2015-06-01 to 2015-09-30")
DCw_Plot.set_title("Minute resolution measuerments of temperature at Deep Cove from 2014-11-01 to 2015-02-28")
JBs_Plot.set_title("Minute resolution measuerments of temperature at James Bay from 2015-06-01 to 2015-09-30")
JBw_Plot.set_title("Minute resolution measuerments of temperature at James Bay from 2014-11-01 to 2015-02-28")

DCs_Plot.set_xlabel("Time")
DCw_Plot.set_xlabel("Time")
JBs_Plot.set_xlabel("Time")
JBw_Plot.set_xlabel("Time")

DCs_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
DCw_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
JBs_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
JBw_Plot.set_ylabel(r"Temperature [$^\circ$ C]")

fig.tight_layout()
plt.show()

<Figure size 864x1152 with 4 Axes>

In [84]:
# Calculate the power spectra
# ===========================

# De-Nan
DCs_DN = DCs[~np.isnan(DCs)]
JBs_DN = DCw[~np.isnan(DCw)]
DCw_DN = JBs[~np.isnan(JBs)]
JBw_DN = JBw[~np.isnan(JBw)]

# Lengths 
[DCs_DN_L, JBs_DN_L, DCw_DN_L, JBw_DN_L] = np.array([len(DCs_DN), len(JBs_DN), len(DCw_DN), len(JBw_DN)]) / (2**2)

# Now calculate
FDCs, GDCs = welch(DCs_DN, DCs_DN_L/2, nperseg=DCs_DN_L,\
            window=sci.signal.windows.hann(int(DCs_DN_L)),\
            noverlap = DCs_DN_L/2, nfft = DCs_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FDCw, GDCw = welch(DCw_DN, JBs_DN_L/2, nperseg=JBs_DN_L,\
            window=sci.signal.windows.hann(int(JBs_DN_L)),\
            noverlap = JBs_DN_L/2, nfft = JBs_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FJBs, GJBs = welch(JBs_DN, DCw_DN_L/2, nperseg=DCw_DN_L,\
            window=sci.signal.windows.hann(int(DCw_DN_L)),\
            noverlap = DCw_DN_L/2, nfft = DCw_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FJBw, GJBw = welch(JBw_DN, JBw_DN_L/2, nperseg=JBw_DN_L,\
            window=sci.signal.windows.hann(int(JBw_DN_L)),\
            noverlap = JBw_DN_L/2, nfft = JBw_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [85]:
# Plot the power spectra
# ======================
fig, (DCs_PS, JBs_PS, DCw_PS, JBw_PS) = plt.subplots(4, 1, figsize=(12, 16))
# plt.loglog(Fx/(2*pi), Gx, label = r'PDS of $x(t)$')
DCs_PS.loglog(FDCs/(2*pi), GDCs, label = r'PDS of $x_f(t)$', linewidth=1)
JBs_PS.loglog(FJBs/(2*pi), GJBs, label = r'PDS of $x_f(t)$', linewidth=1)
DCw_PS.loglog(FDCw/(2*pi), GDCw, label = r'PDS of $x_f(t)$', linewidth=1)
JBw_PS.loglog(FJBw/(2*pi), GJBw, label = r'PDS of $x_f(t)$', linewidth=1)

DCs_PS.set_title("Power spectrum of minute resolution measuerments of temperature at Deep Cove from 2015-06-01 to 2015-09-30")
DCw_PS.set_title("Power spectrum of minute resolution measuerments of temperature at Deep Cove from 2014-11-01 to 2015-02-28")
JBs_PS.set_title("Power spectrum of minute resolution measuerments of temperature at James Bay from 2015-06-01 to 2015-09-30")
JBw_PS.set_title("Power spectrum of minute resolution measuerments of temperature at James Bay from 2014-11-01 to 2015-02-28")

DCs_PS.set_xlabel("Frequency [Hz]")
DCw_PS.set_xlabel("Frequency [Hz]")
JBs_PS.set_xlabel("Frequency [Hz]")
JBw_PS.set_xlabel("Frequency [Hz]")

DCs_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
DCw_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
JBs_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
JBw_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')

fig.tight_layout()
plt.show()

<Figure size 864x1152 with 4 Axes>

### a) Elliptic filters

In [43]:
# Filter functions
# =================

# Bandpass filters 
def EllipBP(CutL, CutH, rp, rs, fs, order, btype='bandstop'):
    fn = fs/2
    low = CutL / fn
    high = CutH / fn
    b, a = ellip(order, rp, rs, [low, high], btype, output='ba')
    return b, a

def EllipFBP(x, CutL, CutH, rp, rs, fs, order, btype='bandstop'):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllipBP(CutL, CutH, rp, rs, fs, order, btype='bandstop', worN=2000):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

# High/Low-pass filters 
def Ellip(Cut, rp, rs, fs, order, btype='highpass'):
    fn = fs/2
    Ws = Cut / fn
    b, a = ellip(order, rp, rs, Ws, btype, output='ba')
    return b, a

def EllipF(x, Cut, rp, rs, fs, order, btype='highpass'):
    b, a = Ellip(CutL, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllip(Cut, rp, rs, fs, order, btype='highpass', worN=2000):
    b, a = Ellip(Cut, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

In [45]:
# Ellipse up x(t)
fig, (f3d1, f3d2) = plt.subplots(2, 1, figsize=(10, 8))

CutL = 10
CutH = 16
order = 4
rp = 0.5
rs = 1
btype = 'highpass'
fs3 = steps3/range3
# f1 = ButterBPF(x, CutL, CutH, fs3, order)

# Plot the filter
b, a, w, h = GraphEllip(CutL, rp, rs, fs3, order, btype)
f3d1.plot(fs3/(2*pi) * w, abs(h), label="Order = {0}".format(order))
# f3d1.plot([0, fs3/2], [sqrt(0.5), sqrt(0.5)], '--', label=r'$\sqrt{0.5}$')
f3d1.set_title('Elliptic {0} filter of order {1}'.format(btype, order))
f3d1.set_xlabel(r'Frequency ($Hz$)')
f3d1.set_ylabel(r'$H(\omega)$')
f3d1.grid(True)
f3d1.legend(loc='best')
f3d1.set_xlim(0, 30)

# Plot filtered x(t)
# f3d2.plot(t3, x1, label=r'$x_1(t)$')
# f3d2.plot(t3, f1, label=r'$x_f(t)$')
# title3b = r'Isolation of $x1=\cos(22\pi t)$ signal from x(t) via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
# f3d2.set_title("\n".join(wrap(title3b, 90)))
# f3d2.set_xlabel(r'Time ($s$)')
# f3d2.set_ylabel(r'$x_f(t)$')
# f3d2.legend(loc=1)
# f3d2.set_xlim(-1, 1)

fig.tight_layout()
plt.show()

<Figure size 720x576 with 2 Axes>

## Question 2: Spatial interpolation 

## Question 3: Interpolation

In [12]:
fig, US_Plot = plt.subplots(1, 1, figsize=(12, 4))

US_Plot.plot(US, linewidth=1)
US_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
US_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
US_Plot.set_title("Hour resolution measuerments of temperature at UVicSCI from 2015-01-01 to 2015-12-31")
US_Plot.set_xlabel("Time")
US_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
plt.xticks(rotation=90)
fig.tight_layout()
plt.show()

<Figure size 864x288 with 1 Axes>